In [1]:
import numpy as np 
import pywt

In [2]:
def get_nabla_f(sigma, y):
    """Returns the gradient of the function f w.r.t to X 
    """
    # Precalculate variables to save time
    psi_y, slices = pywt.coeffs_to_array(pywt.wavedec2(y, 'haar', level=4))
    def nabla_f(X):
        #return - (psi_y - pywt.coeffs_to_array(pywt.wavedec2(X, 'haar', level=4))[0]) / sigma_2
        return - (psi_y - X) / sigma_2
    
    return nabla_f

In [3]:
def proximal(X, theta):
    """Computes the proximal operator parametrized by lambda and theta^T * g of X     
    
    Note:   Our prox operator of x is argmin_{\bar{x}}(\theta * |\bar{x}| + \frac{\bar{x} - x}{2\lambda})
            which is equivalent to argmin_{\bar{x}}(\theta * \lambda * |\bar{x}| + \frac{\bar{x} - x}{2})
            
    The code below represents the hand calculated solution we found for the argmin
    """
    X_bar = np.copy(X)
    for i in range(len(X)):
        for j in range(len(X[0])):
            sign = 1 if X_bar[i, j] > 0 else -1
            X_bar[i, j] = sign * max(0, np.absolute(X_bar[i, j]) - lambd * theta)
    return X_bar

In [4]:
def myula_kernel(X, nabla_f, theta):
    """Returns the new value of X based on the MYULA kernel
    """
    Z = np.random.normal(size=X.shape)
    return X - gamma * nabla_f(X) - gamma * (X - proximal(X, theta))/lambd + np.sqrt(2*gamma) * Z

In [5]:
def projection(a):
    return 0.000001 if a < 0 else a

In [6]:
def sapg_scalar_homogeneous(theta_0, X, nabla_f, N):
    """SAPG Algorithm for scalar theta and alpha positively homogeneous regularizer g"""
    thetas = [theta_0]
    theta = theta_0
    for i in range(N):
        delta = (i+1)**(-beta) / (theta_0 * dim_X**2)
        X = myula_kernel(X, nabla_f, thetas[i])
        theta = projection(theta + delta * (dim_X**2 / theta - np.linalg.norm(X.reshape(-1), ord=1))) # g(x) = |x|_1
        thetas.append(theta)
    return np.mean(thetas), thetas

In [7]:
def generate_data():
    x = np.random.laplace(size=(dim_X, dim_X))
    coeffs = pywt.array_to_coeffs(x, slices, output_format='wavedec2')
    y = pywt.waverec2(coeffs, 'haar')
    
    return x, y

In [8]:
N = 50
beta = 0.8
dim_Y = 256
dim_X = 256
d = dim_Y**2
theta_0 = 0.5

# Get the slices
dummy_coeff = pywt.wavedec2(np.ones((dim_Y ,dim_Y)), 'haar', level=4)
_, slices = pywt.coeffs_to_array(dummy_coeff)

In [9]:
# Define Params for the Algorithm
sigma_2 = 0.01 #SNR = 20 -> sigma_2 = 0.01
L_y = 1/sigma_2
lambd = max(5 * 1/L_y, 2.0) # currently L_y=5
gamma = 0.98 /(L_y + 1/lambd)

In [10]:
_, y = generate_data()
nabla_f = get_nabla_f(sigma_2, y)
X_0 = np.zeros((dim_X, dim_X))
sapg_scalar_homogeneous(theta_0, X_0, nabla_f, N)

(1.070555554679737,
 [0.5,
  2.5364591216935306,
  1.839754737937295,
  1.4590061682937807,
  1.250648299668402,
  1.1389356866577238,
  1.0800002376051476,
  1.0477964863056852,
  1.0296204830554492,
  1.0189194141749613,
  1.0123311912924982,
  1.0081401285684226,
  1.0052302313687318,
  1.0033848855993261,
  1.0022331495155283,
  1.0011685126236731,
  1.0002777972673487,
  0.9996012558410685,
  0.9991212223100562,
  0.998869620993437,
  0.9986704976823816,
  0.998680837121914,
  0.9986377530637027,
  0.9985762192416743,
  0.9984838850248196,
  0.998298342516528,
  0.9982936721967471,
  0.9982613691598008,
  0.9980789026402807,
  0.997961576391599,
  0.9980274615645417,
  0.9979437798971665,
  0.9979058386336492,
  0.9980297414156368,
  0.9980140140470508,
  0.9978668919803351,
  0.9978565060372493,
  0.9978587761079166,
  0.9979788485826067,
  0.9978924594419276,
  0.9979617383496215])

In [11]:
# Define Params for the Algorithm
sigma_2 = 0.001 #SNR = 30 -> sigma_2 = 0.001
L_y = 1/sigma_2
lambd = max(5 * 1/L_y, 2.0) # currently L_y=5
gamma = 0.98 /(L_y + 1/lambd)

In [12]:
_, y = generate_data()
nabla_f = get_nabla_f(sigma_2, y)
X_0 = np.zeros((dim_X, dim_X))
sapg_scalar_homogeneous(theta_0, X_0, nabla_f, N)

(1.0710815365847006,
 [0.5,
  2.536789518079644,
  1.8399099673933348,
  1.4596136055857551,
  1.2509465991509814,
  1.1394086661150624,
  1.0802536531048907,
  1.0482627862476754,
  1.030203656947974,
  1.0194232210387637,
  1.0129061720649835,
  1.0086626553384712,
  1.005888356007752,
  1.0039075174609988,
  1.0025992906524916,
  1.0016439901714402,
  1.000963026320855,
  1.0003479623429934,
  0.9999864258632208,
  0.9996827916225618,
  0.9994748095043758,
  0.999305858454614,
  0.9991569929542166,
  0.9990033448701687,
  0.998896858485755,
  0.9987798794914224,
  0.9987191531576778,
  0.998640119964843,
  0.9986413538366217,
  0.9986040157596988,
  0.9985693497901076,
  0.9985308606393248,
  0.9985160125217926,
  0.9985453687105561,
  0.9985369360962596,
  0.9985077970206131,
  0.9985323283252838,
  0.9985123261536993,
  0.9984968485546474,
  0.9984708471796723,
  0.9985020769915222])

In [13]:
# Define Params for the Algorithm
sigma_2 = 0.0001 #SNR = 40 -> sigma_2 = 0.0001 
L_y = 1/sigma_2
lambd = max(5 * 1/L_y, 2.0) # currently L_y=5
gamma = 0.98 /(L_y + 1/lambd)

In [14]:
_, y = generate_data()
nabla_f = get_nabla_f(sigma_2, y)
X_0 = np.zeros((dim_X, dim_X))
sapg_scalar_homogeneous(theta_0, X_0, nabla_f, N)

(1.0730791081342659,
 [0.5,
  2.5401570879426574,
  1.844114076317111,
  1.4639999189474073,
  1.254916725907453,
  1.1428668572481864,
  1.0832196028019214,
  1.050826044647412,
  1.032498241251455,
  1.021655154206792,
  1.0149175981923888,
  1.0106041040378095,
  1.0077428815432172,
  1.0057598286090002,
  1.0043687062813789,
  1.0033910302395994,
  1.0026355108815204,
  1.0020762854364376,
  1.0016704203361493,
  1.0013638714550268,
  1.0011103671175634,
  1.0009086128401354,
  1.0007645872232216,
  1.0006570069622873,
  1.0005677628034193,
  1.0004757589437918,
  1.0003963941435507,
  1.0003442888540424,
  1.000300642350983,
  1.0002671820291245,
  1.0002394102384335,
  1.000211684912781,
  1.00018461334595,
  1.0001650450132347,
  1.0001612195026055,
  1.0001430489713017,
  1.000128055411781,
  1.0001211801308316,
  1.0001113129248138,
  1.0001054658383837,
  1.0000958476637456])